In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import requests

In [6]:
engine=create_engine("sqlite+pysqlite:///:memory:")

In [7]:
df=pd.read_csv('udemy_courses.csv')
df.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5,2017-01-18T20:58:58Z,Business Finance
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39.0,2017-03-09T16:34:20Z,Business Finance
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5,2016-12-19T19:26:30Z,Business Finance
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3.0,2017-05-30T20:07:24Z,Business Finance
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2.0,2016-12-13T14:57:18Z,Business Finance


In [8]:
#load data frame into udemy record table in the data base
df.to_sql('udemy_record', engine)
#connect to engine
db_engine=engine.connect()
print('record inserted successfully and engine connected')

record inserted successfully and engine connected


In [9]:
def query_db(query:str,db_conn:sqlalchemy.engine.base.Connection)->pd.DataFrame:
    sql_df=pd.read_sql(query,db_conn)
    return sql_df

In [12]:
df=query_db("SELECT * FROM udemy_record LIMIT 10", db_engine)
df

,index,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
0,0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,1,200,2147,23,51,All Levels,1.5,2017-01-18T20:58:58Z,Business Finance
1,1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,1,75,2792,923,274,All Levels,39.0,2017-03-09T16:34:20Z,Business Finance
2,2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,1,45,2174,74,51,Intermediate Level,2.5,2016-12-19T19:26:30Z,Business Finance
3,3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,1,95,2451,11,36,All Levels,3.0,2017-05-30T20:07:24Z,Business Finance
4,4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,1,200,1276,45,26,Intermediate Level,2.0,2016-12-13T14:57:18Z,Business Finance
5,5,192870,Trading Penny Stocks: A Guide for All Levels I...,https://www.udemy.com/trading-penny-stocks-a-g...,1,150,9221,138,25,All Levels,3.0,2014-05-02T15:13:30Z,Business Finance
6,6,739964,Investing And Trading For Beginners: Mastering...,https://www.udemy.com/investing-and-trading-fo...,1,65,1540,178,26,Beginner Level,1.0,2016-02-21T18:23:12Z,Business Finance
7,7,403100,"Trading Stock Chart Patterns For Immediate, Ex...",https://www.udemy.com/trading-chart-patterns-f...,1,95,2917,148,23,All Levels,2.5,2015-01-30T22:13:03Z,Business Finance
8,8,476268,Options Trading 3 : Advanced Stock Profit and ...,https://www.udemy.com/day-trading-stock-option...,1,195,5172,34,38,Expert Level,2.5,2015-05-28T00:14:03Z,Business Finance
9,9,1167710,The Only Investment Strategy You Need For Your...,https://www.udemy.com/the-only-investment-stra...,1,200,827,14,15,All Levels,1.0,2017-04-18T18:13:32Z,Business Finance


In [13]:
query_db("""SELECT course_title, MAX(num_subscribers) AS high_subscriber, strftime('%m',published_timestamp)AS month
FROM udemy_record 
WHERE is_paid=1 
GROUP BY strftime('%m',published_timestamp) """, db_engine)

,course_title,high_subscriber,month
0,Learn Web Designing & HTML5/CSS3 Essentials in...,43285,01
1,Angular 4 (formerly Angular 2) - The Complete ...,73783,02
2,The Complete Web Developer Course 2.0,114512,03
3,Web Programming with Python,35267,04
4,The Complete Web Developer Masterclass: Beginn...,37141,05
5,Learning Dynamic Website Design - PHP MySQL an...,47886,06
6,The Complete HTML & CSS Course - From Novice T...,57422,07
7,Pianoforall - Incredible New Way To Learn Pian...,75499,08
8,Learn and Understand AngularJS,59361,09
9,Learn Javascript & JQuery From Scratch,84897,10


In [15]:
query_db("""SELECT course_title, MAX(num_subscribers * price) AS high_sales, strftime('%m',published_timestamp) AS month
FROM udemy_record
WHERE is_paid=1
GROUP BY strftime('%m', published_timestamp)""", db_engine)

,course_title,high_sales,month
0,Master the MEAN Stack - Learn By Example,5030805,01
1,Angular 4 (formerly Angular 2) - The Complete ...,14018770,02
2,The Complete Web Developer Course 2.0,22902400,03
3,"Coding for Entrepreneurs: Learn Python, Django...",4565340,04
4,The Complete Web Developer Masterclass: Beginn...,7242495,05
5,Photoshop for Entrepreneurs - Design 11 Practi...,7257600,06
6,The Complete HTML & CSS Course - From Novice T...,11197290,07
7,Pianoforall - Incredible New Way To Learn Pian...,15099800,08
8,Complete PHP Course With Bootstrap3 CMS System...,10789740,09
9,The Complete Ruby on Rails Developer Course,6567015,10
